# Init

In [78]:
!sudo apt update
!pip install pyspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,648 kB]
Get:14 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,450 kB]
Fetched 4,486 kB in

In [73]:
# Добавляем Hive таблицу employee
from pyspark.sql import SparkSession
spark = SparkSession \
       .builder \
       .master('local[*]') \
       .appName("Our First Spark Example") \
       .enableHiveSupport() \
       .getOrCreate()
spark.read.parquet('employee.parquet').repartition(1).write.saveAsTable("employee")
spark.stop()

# Старт сессии и импорт

Импортируем необходимые библиотеки

Документация PySpark: https://spark.apache.org/docs/latest/api/python/index.html

Документация PySpark SQL Functions: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html

In [ ]:
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

In [27]:
# Создание спарк сессии
spark = SparkSession \
       .builder \
       .master('local[*]') \
       .appName("Our First Spark Example") \
       .enableHiveSupport() \
       .getOrCreate()

spark

#Input Data

1) Создать собственный DataFrame

2) Считать из Hive-таблицы

3) Считать из файла любого формата

In [37]:
# Создаем собственный DataFrame
sample_data = [
    {"name": "John D.",   "age": 30},
    {"name": "Alice G.",  "age": 25},
    {"name": "Bob T.",    "age": 35},
    {"name": "EvecA.",    "age": 28}
    ]

sample_df = spark.createDataFrame(sample_data)
sample_df.show()

+---+--------+
|age|    name|
+---+--------+
| 30| John D.|
| 25|Alice G.|
| 35|  Bob T.|
| 28|  EvecA.|
+---+--------+



Читает DataFrame из Hive-таблицы

In [85]:
# Смотрим, какие таблицы есть в Hive metastore
spark.sql("show tables").show()

+---------+-----------+-----------+
|namespace|  tableName|isTemporary|
+---------+-----------+-----------+
|  default|   employee|      false|
|         |employee_df|       true|
+---------+-----------+-----------+



In [87]:
employee_h_df = spark.read.table("employee")
employee_h_df

DataFrame[Id: int, LastName: string, FirstName: string, MiddleName: string, Gender: string, Age: int, Department: string, Position: string, Grade: string, HireDate: date, OfferAmount: double, Education: string, CurrentStatus: string]

In [88]:
employee_h_df.show(5)

+---+--------+---------+------------+------+---+----------+--------------------+------+----------+-----------+-------------------+-------------+
| Id|LastName|FirstName|  MiddleName|Gender|Age|Department|            Position| Grade|  HireDate|OfferAmount|          Education|CurrentStatus|
+---+--------+---------+------------+------+---+----------+--------------------+------+----------+-----------+-------------------+-------------+
|  2| Абрамов|   Анжела|   Тихонович|Female| 54|        IT|Продуктовый аналитик|Стажер|2015-12-20|  101972.96|       Магистратура|      Активен|
|  3| Абрамов|Харлампий|   Елисеевич|Female| 38|        IT|  Разработчик систем|Стажер|2019-09-30|   88074.95|Среднее специальное|       Уволен|
|  4| Абрамов|   Ерофей|Валентинович|  Male| 34| Analytics|Разработчик баз д...|Junior|2024-03-08|  120694.35|Среднее специальное|      Активен|
|  5|  Авдеев|   Трофим|   Антонович|Female| 58|   Product|     Проект менеджер|Junior|2018-10-14|  121063.82|       Магистратура|

Читает DataFrame из файла parquet

In [80]:
# Смотрим, какие файлы лежат в нашей директории (linux-командой)
!ls -la | grep employee

drwxr-xr-x 2 root root     4096 Oct 18 21:47 employee.csv
-rw-r--r-- 1 root root   172147 Oct 18 14:57 employee_data.csv
drwxr-xr-x 2 root root     4096 Oct 18 21:46 employee.parquet


In [81]:
# Читаем parquet-файл
employee_p_df = spark.read.parquet('employee.parquet')
employee_p_df

DataFrame[Id: int, LastName: string, FirstName: string, MiddleName: string, Gender: string, Age: int, Department: string, Position: string, Grade: string, HireDate: date, OfferAmount: double, Education: string, CurrentStatus: string]

In [83]:
employee_p_df.show(5)

+---+--------+---------+------------+------+---+----------+--------------------+------+----------+-----------+-------------------+-------------+
| Id|LastName|FirstName|  MiddleName|Gender|Age|Department|            Position| Grade|  HireDate|OfferAmount|          Education|CurrentStatus|
+---+--------+---------+------------+------+---+----------+--------------------+------+----------+-----------+-------------------+-------------+
|  1| Абрамов|   Самсон|Валерьянович|Female| 36| Analytics|     Аналитик данных|Стажер|2024-06-14|   89178.63|       Магистратура|       Уволен|
|  2| Абрамов|   Анжела|   Тихонович|Female| 54|        IT|Продуктовый аналитик|Стажер|2015-12-20|  101972.96|       Магистратура|      Активен|
|  3| Абрамов|Харлампий|   Елисеевич|Female| 38|        IT|  Разработчик систем|Стажер|2019-09-30|   88074.95|Среднее специальное|       Уволен|
|  4| Абрамов|   Ерофей|Валентинович|  Male| 34| Analytics|Разработчик баз д...|Junior|2024-03-08|  120694.35|Среднее специальное|

Читает DataFrame из файла csv

In [79]:
# Смотрим, какие файлы лежат в нашей директории (linux-командой)
!ls -la | grep employee

drwxr-xr-x 2 root root     4096 Oct 18 21:47 employee.csv
-rw-r--r-- 1 root root   172147 Oct 18 14:57 employee_data.csv
drwxr-xr-x 2 root root     4096 Oct 18 21:46 employee.parquet


In [29]:
# Читаем первые 10 строк файла (linux-командой)
!head employee_data.csv

Id,LastName,FirstName,MiddleName,Gender,Age,Department,Position,Grade,HireDate,OfferAmount,Education,CurrentStatus
3236,Брагина,Алина,Гертрудович,Male,22,Analytics,Продукт менеджер,Middle,2022-07-30,262810.4,Среднее специальное,Уволен
1721,Савина,Ермил,Сергеевна,Male,48,Data,Аналитик данных,Senior,2019-11-27,301038.97,Среднее специальное,Активен
1026,Игнатова,Харлампий,Зиновьевич,Female,56,IT,Разработчик баз данных,Junior,2022-12-21,115378.85,Среднее специальное,Уволен
8546,Антонова,Валентина,Игоревна,Male,52,Analytics,Разработчик баз данных,Middle,2024-08-11,292886.23,Магистратура,Уволен
1820,Алексеева,Карл,Гордеевич,Male,45,Analytics,Системный аналитик,Middle,2015-11-13,203323.22,Высшее,Активен
6780,Владимиров,Пахом,Макаровна,Female,45,IT,Аналитик данных,Junior,2023-04-10,106603.51,Магистратура,Активен
9693,Фомичева,Борислав,Борисовна,Male,41,Product,Продуктовый аналитик,Senior,2022-04-09,318778.62,Среднее специальное,Активен
2642,Ермаков,Галактион,Глебович,Male,30,IT,Проект менеджер

In [30]:
# Определяем схему для витрины "Сотрудники"
schema = StructType([
    StructField("Id", IntegerType(), False),  # Уникальный идентификатор
    StructField("LastName", StringType(), False),  # Фамилия
    StructField("FirstName", StringType(), False),  # Имя
    StructField("MiddleName", StringType(), True),  # Отчество
    StructField("Gender", StringType(), False),  # Пол
    StructField("Age", IntegerType(), False),  # Возраст
    StructField("Department", StringType(), False),  # Отдел
    StructField("Position", StringType(), False),  # Должность
    StructField("Grade", StringType(), False),  # Грейд
    StructField("HireDate", DateType(), False),  # Дата найма
    StructField("OfferAmount", DoubleType(), False),  # Сумма оффера
    StructField("Education", StringType(), False),  # Образование
    StructField("CurrentStatus", StringType(), False)  # Статус сотрудника
])

# Читаем данные из CSV файла
employee_df = spark.read.csv('employee.csv', schema=schema, header=True)
employee_df

DataFrame[Id: int, LastName: string, FirstName: string, MiddleName: string, Gender: string, Age: int, Department: string, Position: string, Grade: string, HireDate: date, OfferAmount: double, Education: string, CurrentStatus: string]

In [64]:
# Показать схему DataFrame
employee_df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- LastName: string (nullable = true)
 |-- FirstName: string (nullable = true)
 |-- MiddleName: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Department: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Grade: string (nullable = true)
 |-- HireDate: date (nullable = true)
 |-- OfferAmount: double (nullable = true)
 |-- Education: string (nullable = true)
 |-- CurrentStatus: string (nullable = true)



In [34]:
# Вывести 5 строк (не сокращая)
employee_df.show(5, False)

+----+---------+---------+-----------+------+---+----------+----------------------+------+----------+-----------+-------------------+-------------+
|Id  |LastName |FirstName|MiddleName |Gender|Age|Department|Position              |Grade |HireDate  |OfferAmount|Education          |CurrentStatus|
+----+---------+---------+-----------+------+---+----------+----------------------+------+----------+-----------+-------------------+-------------+
|3236|Брагина  |Алина    |Гертрудович|Male  |22 |Analytics |Продукт менеджер      |Middle|2022-07-30|262810.4   |Среднее специальное|Уволен       |
|1721|Савина   |Ермил    |Сергеевна  |Male  |48 |Data      |Аналитик данных       |Senior|2019-11-27|301038.97  |Среднее специальное|Активен      |
|1026|Игнатова |Харлампий|Зиновьевич |Female|56 |IT        |Разработчик баз данных|Junior|2022-12-21|115378.85  |Среднее специальное|Уволен       |
|8546|Антонова |Валентина|Игоревна   |Male  |52 |Analytics |Разработчик баз данных|Middle|2024-08-11|292886.23  

In [31]:
# Вывести основные метрики по Int атрибутам
employee_df.select("Id", "Age", "OfferAmount").describe().show()

+-------+------------------+-----------------+-----------------+
|summary|                Id|              Age|      OfferAmount|
+-------+------------------+-----------------+-----------------+
|  count|              1000|             1000|             1000|
|   mean|          5455.959|           40.955|     278302.03539|
| stddev|2574.7894893045586|11.05247176572427|189698.3213869978|
|    min|              1008|               22|         80065.42|
|    max|              9998|               60|         798408.5|
+-------+------------------+-----------------+-----------------+



In [42]:
# Переводим DF в temp view (для обращения через spark.sql)
employee_df.createOrReplaceTempView("employee_df")

In [ ]:
# Посмотрим, появилась ли таблица в Hive metastore
spark.sql("show tables").show()

In [43]:
# Выведем select всех записей
spark.sql("select * from employee").show()

+----+-----------+-----------+------------+------+---+----------+--------------------+------+----------+-----------+-------------------+-------------+
|  Id|   LastName|  FirstName|  MiddleName|Gender|Age|Department|            Position| Grade|  HireDate|OfferAmount|          Education|CurrentStatus|
+----+-----------+-----------+------------+------+---+----------+--------------------+------+----------+-----------+-------------------+-------------+
|3236|    Брагина|      Алина| Гертрудович|  Male| 22| Analytics|    Продукт менеджер|Middle|2022-07-30|   262810.4|Среднее специальное|       Уволен|
|1721|     Савина|      Ермил|   Сергеевна|  Male| 48|      Data|     Аналитик данных|Senior|2019-11-27|  301038.97|Среднее специальное|      Активен|
|1026|   Игнатова|  Харлампий|  Зиновьевич|Female| 56|        IT|Разработчик баз д...|Junior|2022-12-21|  115378.85|Среднее специальное|       Уволен|
|8546|   Антонова|  Валентина|    Игоревна|  Male| 52| Analytics|Разработчик баз д...|Middle|2

In [45]:
# Выведем ФИО и позицию только активных сотрудников
# employee_df.filter("CurrentStatus = 'Активен'").select("LastName", "FirstName", "MiddleName", "Position").show(10, False)
spark.sql("""
select
  LastName,
  FirstName,
  MiddleName,
  Position
from employee
where CurrentStatus = 'Активен'
          """).show(10, False)

+----------+-----------+-----------+--------------------+
|LastName  |FirstName  |MiddleName |Position            |
+----------+-----------+-----------+--------------------+
|Савина    |Ермил      |Сергеевна  |Аналитик данных     |
|Алексеева |Карл       |Гордеевич  |Системный аналитик  |
|Владимиров|Пахом      |Макаровна  |Аналитик данных     |
|Фомичева  |Борислав   |Борисовна  |Продуктовый аналитик|
|Агафонова |Ермил      |Альбертовна|Системный инженер   |
|Казаков   |Марина     |Михайловна |Разработчик систем  |
|Рогов     |Нестор     |Леонидовна |Системный аналитик  |
|Гусева    |Фирс       |Филатович  |Аналитик данных     |
|Рыбаков   |Флорентин  |Юльевна    |Разработчик систем  |
|Полякова  |Максимильян|Аркадьевна |Разработчик систем  |
+----------+-----------+-----------+--------------------+
only showing top 10 rows



In [51]:
# Выведем количество уволенных и активных сотрудников
# employee_df.groupBy("CurrentStatus").agg(F.count("LastName").alias("Count")).show()
spark.sql("""
select
  CurrentStatus,
  count(*) as Count
from employee
group by CurrentStatus
          """).show()

+-------------+-----+
|CurrentStatus|Count|
+-------------+-----+
|      Активен|  504|
|       Уволен|  496|
+-------------+-----+



In [63]:
# Вывести среднюю зп (OfferAmount) у Активных сотрудников в зависимости от Grade (округлить до 2 знаков после запятой)
spark.sql("""

          """).show()

+------+-------------+
| Grade|          Avg|
+------+-------------+
|Junior|125800.04 руб|
|  Lead|598093.88 руб|
|Middle|219088.67 руб|
|Senior|322294.19 руб|
|Стажер| 99122.71 руб|
+------+-------------+



In [ ]:
# 1) Посчитать средний возраст всех сотрудников.
spark.sql("""

          """).show()

In [ ]:
# 2) Посчитать количество мужчин и женщин в компании.
spark.sql("""

          """).show()

In [ ]:
# 3) Посчитать распределение сотрудников по отделам (Department).
spark.sql("""

          """).show()

In [ ]:
# 4*) Оценить уровень текучести кадров, посчитав процент уволенных сотрудников.
spark.sql("""

          """).show()

In [93]:
# 5) Определить, как долго сотрудники работают в компании, используя разницу между текущей датой и датой их найма.
# from pyspark.sql.functions import current_date, datediff
spark.sql("""

          """).show()

+----------------------------------+
|datediff(current_date(), HireDate)|
+----------------------------------+
|                              3225|
|                              1845|
|                               224|
|                              2196|
|                              3298|
|                              3023|
|                              2927|
|                                34|
|                              1504|
|                               816|
|                              3278|
|                              3367|
|                              3301|
|                              3481|
|                              3530|
|                              2505|
|                              3376|
|                              1452|
|                              2532|
|                               926|
+----------------------------------+
only showing top 20 rows



In [ ]:
# 6) Определить средний доход специалистов в зависимости от уровня образования
spark.sql("""

          """).show()

In [ ]:
# 7) Посчитать среднюю продолжительность работы сотрудников в компании (с учетом даты найма).
spark.sql("""

          """).show()

In [ ]:
# 8) Преобразовать CurrentStatus в Int (Уволен = 0, Активен = 1)
spark.sql("""

          """).show()

# Новый DataFrame (Attendance)

In [104]:
!ls -la | grep attendance

drwxr-xr-x 2 root root     4096 Oct 18 22:22 attendance.parquet


In [ ]:
# Считать DataFrame attendance.parquet


In [ ]:
# 9) Посчитать среднее количество отработанных часов для всех сотрудников.
spark.sql("""

          """).show()

In [ ]:
# 10) Посчитать долю сотрудников, которые в среднем работают больше 8 часов в день.
spark.sql("""

          """).show()

In [ ]:
df.groupBy("age").count().explain(True)

== Parsed Logical Plan ==
'Aggregate ['age], ['age, count(1) AS count#41L]
+- LogicalRDD [age#0L, name#1], false

== Analyzed Logical Plan ==
age: bigint, count: bigint
Aggregate [age#0L], [age#0L, count(1) AS count#41L]
+- LogicalRDD [age#0L, name#1], false

== Optimized Logical Plan ==
Aggregate [age#0L], [age#0L, count(1) AS count#41L]
+- Project [age#0L]
   +- LogicalRDD [age#0L, name#1], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[age#0L], functions=[count(1)], output=[age#0L, count#41L])
   +- Exchange hashpartitioning(age#0L, 200), ENSURE_REQUIREMENTS, [plan_id=86]
      +- HashAggregate(keys=[age#0L], functions=[partial_count(1)], output=[age#0L, count#45L])
         +- Project [age#0L]
            +- Scan ExistingRDD[age#0L,name#1]



Оконные функции

In [ ]:
from pyspark.sql.window import Window

windowSpec = Window.partitionBy("course").orderBy("income")
df1.withColumn("rn", F.row_number().over(windowSpec)).show()

Udf-функции

In [ ]:
@udf(returnType=StringType())
def upperCase(str):
    return str.upper()

df.withColumn("Cureated Name", upperCase(col("Name"))).show(truncate=False)

When-конструктор

In [ ]:
(covid_df
  .select('date', ‘location',
    when(col('total_cases') > 10000, 'Red zone')
    .when((col('total_cases') <= 10000) & (col('total_cases') > 5000), 'Yellow zone')
    .otherwise('Green zone').alias('zone'))
  .show())

In [26]:
spark.stop()